# Loading the test dataset and LGBM model

In [18]:
!pip3 install -q -U lightgbm
!pip3 install -q -U xgboost
!pip3 install -q -U catboost

In [19]:
# Loading the python pacakges and Libraries

import os
import numpy as np
import pandas as pd
import joblib
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import gc
import seaborn as sns
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import kurtosis
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

## Loading the test dataset

In [20]:
# Loading the test_merged dataset
test_merged = pd.read_csv('./final_test_data.csv')

# Checking the test_merged dataset
test_merged.head(10)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,PREV_LAST24M_DAYS_LAST_DUE_1ST_VERSION_MAX,PREV_LAST24M_DAYS_LAST_DUE_1ST_VERSION_MEAN,PREV_LAST24M_APPLICATION_CREDIT_DIFF_MIN,PREV_LAST24M_APPLICATION_CREDIT_RATIO_MIN,PREV_LAST24M_APPLICATION_CREDIT_RATIO_MAX,PREV_LAST24M_APPLICATION_CREDIT_RATIO_MEAN,PREV_LAST24M_NAME_CONTRACT_TYPE_Consumer loans_MEAN,PREV_LAST24M_NAME_CONTRACT_TYPE_Cash loans_MEAN,PREV_LAST24M_NAME_CONTRACT_TYPE_Revolving loans_MEAN,TARGET
0,100001,0,1,0,135000.0,568800.0,20560.5,450000.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005,0,0,0,99000.0,222768.0,17370.0,180000.0,0,0,...,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,1.000000,0.000000,NaN
2,100013,0,0,1,202500.0,663264.0,69777.0,630000.0,-1,0,...,-92.0,-92.0,-1777.5,0.926648,0.926648,0.926648,0.500000,0.500000,0.000000,NaN
3,100028,0,1,0,315000.0,1575000.0,49018.5,1575000.0,0,0,...,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,1.000000,0.000000,NaN
4,100038,0,0,1,180000.0,625500.0,32067.0,625500.0,0,0,...,NaN,NaN,-53995.5,0.893813,0.893813,0.893813,0.000000,1.000000,0.000000,NaN
5,100042,0,1,1,270000.0,959688.0,34600.5,810000.0,0,1,...,401.0,401.0,-84289.5,0.919861,0.919861,0.919861,0.000000,1.000000,0.000000,NaN
6,100057,0,0,1,180000.0,499221.0,22117.5,373500.0,0,0,...,182.0,130.0,-10174.5,0.889868,1.249986,1.069927,1.000000,0.000000,0.000000,NaN
7,100065,0,0,0,166500.0,180000.0,14220.0,180000.0,0,0,...,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,1.000000,0.000000,NaN
8,100066,0,1,0,315000.0,364896.0,28957.5,315000.0,0,1,...,1041.0,221.0,-95040.0,0.825627,1.000000,0.926918,0.250000,0.625000,0.125000,NaN
9,100067,0,1,1,162000.0,45000.0,5337.0,45000.0,1,0,...,172.0,-30.0,-83160.0,0.844024,1.250066,0.993298,0.176471,0.705882,0.117647,NaN


In [21]:
# Loading the scaled test data numpy array dataset
import numpy as np

# Load the scaled test data from the NumPy binary file
test = np.load('./test.npy')

## Loading the LGBM Classifier Model

In [22]:
# Loading the LGBM Classifier Model
# lgbmc = joblib.load('./hcdr_final_lgbmclass_model.joblib')
lgbmc = joblib.load('./best_model.joblib')
print(type(lgbmc))

<class 'lightgbm.sklearn.LGBMClassifier'>


# Prediction

In [23]:
lgbm_pred = lgbmc.predict_proba(test)[:, 1]

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [24]:
submit = test_merged[['SK_ID_CURR']]
submit['TARGET'] = lgbm_pred

/var/folders/1h/1n2c6vgn26s1qd7nj20z38hm0000gp/T/ipykernel_17139/2591123886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['TARGET'] = lgbm_pred


In [25]:
submit.head()

,SK_ID_CURR,TARGET
0,100001,0.042176
1,100005,0.163820
2,100013,0.035550
3,100028,0.053229
4,100038,0.159516


# Submission

In [26]:
submit.to_csv('submission01.csv', index = False, header=True)